In [ ]:
%load_ext autoreload

# (*) 以数据集形式同步读取AOP和COP，默认做对齐和ISP处理

- TianmoucDataReader 的測試功能
    - ## [输入]
    - aop_denoise, 是否開啓AOP去噪
    - aop_denoise_args = denoise_defualt_args(), 去噪默認參數
        

## 测试数据路径

In [ ]:
%autoreload
import sys,os
import numpy as np
import torch
import matplotlib.pyplot as plt
from tianmoucv.isp import SD2XY
from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F
import cv2

train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]
val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []
print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
        
all_data = valdata + traindata

# 对比去噪和不去噪的结果

In [ ]:
%autoreload

from tianmoucv.isp import vizDiff
from tianmoucv.data.denoise_utils import denoise_defualt_args

aim = 4 # 读取的目标COP帧的帧号
N = 1   # read continue N frames （连续读取N帧COP以及对应的AOP）

# 这些形式都可以读取数据
datapath = "/data/lyh/tianmoucData/tianmoucReconDataset/train/normal/train_cross2/"
datapath = [train,val]
datapath = valdata + traindata
key_list = ['train_cross2']

for key in key_list:

    # 默认关闭去噪
    dataset = TianmoucDataReader(datapath,matchkey=key,aop_denoise = False,camera_idx=0)

    # 开启去噪
    defualt_args = denoise_defualt_args()
    denoised_dataset = TianmoucDataReader(datapath,matchkey=key,aop_denoise = True, aop_denoise_args = defualt_args, camera_idx=0)
    
    img_list = []
    for index in range(aim,min(aim+1,len(dataset))):
        sample = dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0

            tsd_rgb = tsdiff[:,i,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)
            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()

    print('去噪结果')
    for index in range(aim,min(aim+1,len(denoised_dataset))):
        sample = denoised_dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0

            tsd_rgb = tsdiff[:,i,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)
            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()
        